#  VacationPy
***
### Note 
- Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

In [3]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

### Store Part I results into DataFrame
- Load the csv exported in Part I to a DataFrame

In [11]:
# read csv and create a dataframe
weather_df = pd.read_csv('../Output_files/cities_weather.csv')
weather_df['City'] = weather_df['City'].str.title()
weather_df.head()

,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date
0,Busselton,-33.6500,115.3333,61.00,75,57,4.00,AU,1612916456
1,Kamenka,51.3223,42.7678,12.00,90,100,10.45,RU,1612916148
2,Port Alfred,-7.0349,29.7638,73.94,73,100,4.00,CD,1612916114
3,Broome,42.2506,-75.8330,21.20,73,90,8.05,US,1612916297
4,Maragogi,-9.0122,-35.2225,78.40,77,27,6.02,BR,1612916636


### Humidity Heatmap
- Configure gmaps.
- Use the Lat and Lng as locations and Humidity as the weight.
- Add Heatmap layer to map

In [ ]:
# configure gmaps
gmaps.configure(api_key=gkey)

In [8]:
# use Lat and Long as locations, Humidity as the weight
location = weather_df[['Lat', 'Lng']].astype(float)
humidity_w = weather_df['Humidity (%)']

In [18]:
# customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# plot heatmap
fig = gmaps.figure(layout=figure_layout)

# create heat layer
heat_layer = gmaps.heatmap_layer(location, weights=humidity_w, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
# add layer
fig.add_layer(heat_layer)

# save png file
plt.savefig('../Images/vacationheatmap.png')

# display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

<Figure size 432x288 with 0 Axes>

### Create new Data Frame fitting weather criteria
- Narrow down the cities to fit weather conditions.
- Drop any rows will null values.

In [12]:
# narrow down the cities to fit weather conditions
# drop any rows with null values
reduced_weather_df = weather_df.loc[(weather_df['Max Temp (F)'] <= 80) & (weather_df['Max Temp (F)'] >= 70)
                                   & (weather_df['Wind Speed (MPH)'] < 10) & (weather_df['Cloudiness (%)'] == 0)].dropna()

reduced_weather_df

,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date
63,Ancud,-41.8697,-73.8203,75.20,43,0,4.61,CL,1612916180
72,Codrington,-38.2667,141.9667,74.89,49,0,6.49,AU,1612916171
104,Linguere,15.3953,-15.1193,71.60,30,0,4.61,SN,1612916484
117,Ranong,9.9658,98.6348,75.20,94,0,2.30,TH,1612916488
119,Springbok,-29.6643,17.8865,71.40,39,0,3.87,ZA,1612916489
127,Saint-Philippe,-21.3585,55.7679,77.00,69,0,5.75,RE,1612916491
151,Maceio,-9.6658,-35.7353,75.20,88,0,4.61,BR,1612916673
175,San Cristobal,13.3034,-4.8956,76.87,18,0,8.25,ML,1612915885
176,Quelimane,-17.8786,36.8883,78.80,78,0,4.61,MZ,1612916678
183,Diapaga,12.0708,1.7889,74.16,15,0,6.49,BF,1612916505


### Hotel Map
- Store into variable named hotel_df.
- Add a "Hotel Name" column to the DataFrame.
- Set parameters to search for hotels with 5000 meters.
- Hit the Google Places API for each city's coordinates.
- Store the first Hotel result into the DataFrame.Plot markers on top of the heatmap.

In [13]:
# Store into variable named hotel_df.
hotel_df = reduced_weather_df

# Add a "Hotel Name" column to the DataFrame.
hotel_df['Hotel Name'] = ""
hotel_df

,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date,Hotel Name
63,Ancud,-41.8697,-73.8203,75.20,43,0,4.61,CL,1612916180,
72,Codrington,-38.2667,141.9667,74.89,49,0,6.49,AU,1612916171,
104,Linguere,15.3953,-15.1193,71.60,30,0,4.61,SN,1612916484,
117,Ranong,9.9658,98.6348,75.20,94,0,2.30,TH,1612916488,
119,Springbok,-29.6643,17.8865,71.40,39,0,3.87,ZA,1612916489,
127,Saint-Philippe,-21.3585,55.7679,77.00,69,0,5.75,RE,1612916491,
151,Maceio,-9.6658,-35.7353,75.20,88,0,4.61,BR,1612916673,
175,San Cristobal,13.3034,-4.8956,76.87,18,0,8.25,ML,1612915885,
176,Quelimane,-17.8786,36.8883,78.80,78,0,4.61,MZ,1612916678,
183,Diapaga,12.0708,1.7889,74.16,15,0,6.49,BF,1612916505,


In [14]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# set up a parameters dictionary & search for hotels with 5000 meters
params = {
    'keyword': 'Hotel',
    'radius': 5000,                     
    'type': 'lodging',
    'key': gkey
}

# use interrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    try:
        # get lats and lng from df
        lat = row['Lat']
        lng = row['Lng']
    
        # add keyword to params dict
        params['location'] = f'{lat}, {lng}'
        
        # assemble url and make API request & # Hit the Google Places API for each city's coordinates.
        response = requests.get(base_url, params=params).json()
        
        # extract & # Store the first Hotel result into the DataFrame.
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    
    # 
    except IndexError: 
        
        hotel_df.loc[index, 'Hotel Name'] = 'NaN'

hotel_df

,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (MPH),Country,Date,Hotel Name
63,Ancud,-41.8697,-73.8203,75.20,43,0,4.61,CL,1612916180,Panamericana Hotel Ancud
72,Codrington,-38.2667,141.9667,74.89,49,0,6.49,AU,1612916171,Codrington Gardens Bed And Breakfast
104,Linguere,15.3953,-15.1193,71.60,30,0,4.61,SN,1612916484,Centre Amadou Seck - Linguere
117,Ranong,9.9658,98.6348,75.20,94,0,2.30,TH,1612916488,Sook Hotel
119,Springbok,-29.6643,17.8865,71.40,39,0,3.87,ZA,1612916489,Annie's Cottage - best b&b in Springbok
127,Saint-Philippe,-21.3585,55.7679,77.00,69,0,5.75,RE,1612916491,Les Embruns Du Baril
151,Maceio,-9.6658,-35.7353,75.20,88,0,4.61,BR,1612916673,Jatiúca Hotel & Resort
175,San Cristobal,13.3034,-4.8956,76.87,18,0,8.25,ML,1612915885,Hotel Santoro
176,Quelimane,-17.8786,36.8883,78.80,78,0,4.61,MZ,1612916678,Hotel Elite
183,Diapaga,12.0708,1.7889,74.16,15,0,6.49,BF,1612916505,NaN


In [15]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
 # Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)
fig.add_layer(markers)

# save png of map with markers
plt.savefig('../Images/hotel_vacationheatmap.png')

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

<Figure size 432x288 with 0 Axes>